In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import sys
sys.path.append('../')
import os
from dotenv import load_dotenv

import numpy as np

from src.models.ising_model import *
from src.utils import *
# from src.corrector import *

from src.circuits.pfs_qcircs import *
from src.circuits.cpfs_qcircs import *
from src.circuits.ising_model_qcircs import *
from src.circuits.average_infidelity_qcircs import *

from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime.fake_provider import FakeQuebec
from qiskit_aer import AerSimulator

import pandas as pd

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

## Hardware implementation

For hardware implementation first we need to setup access to quantum hardware. Specify the following parameters to get access to IBM quantum hardware.

In [4]:
token = #add your token here

service = QiskitRuntimeService(
    channel= #add channel here; channel used is: "ibm_quantum",
    instance= #add instance here; instance used is: 'pinq-quebec-hub/univ-toronto/matterlab',
    token=token)

#backend = service.least_busy(operational=True, simulator=False)
backend = service.backend(name="ibm_quebec") # add your backend here; backend used is: "ibm_quebec"

We now run the average infidility circuit on quantum hardware for a speficed system with variying parameters as follows.

In [5]:
pparam = 0.1
n, J, h = 2, pparam, 1 #r is number of qubits (nqubits)
H, Hxx, Hz = ising_qubit_hamiltonian(n, J, h)
A, B = Hz, Hxx

num_shots = 10**5

time_ticks = np.linspace(0.1,1,10)

list_avg_infid_PF1 = []
list_std_infid_PF1 = []
list_avg_infid_PF2 = []
list_std_infid_PF2 = []

list_avg_infid_CPF1 = []
list_std_infid_CPF1 = []
list_avg_infid_CPF2 = []
list_std_infid_CPF2 = []

for t in time_ticks:
    exactU = exact_evolution_ising_qcirc(J, h, n, t)
    r = 10 #r is number of stepts (nsteps)
    tau = t/r

    UPF1 = PF_qcirc(1, A, B, tau, ppart='Hxx', reps=r)
    UPF2 = PF_qcirc(2, A, B, tau, ppart='Hxx', reps=r)

    UCPF1 = CPF_symp_qcirc(1, A, B, tau, ppart='Hxx', reps=r)
    UCPF2 = CPF_symp_qcirc(2, A, B, tau, ppart='Hxx', reps=r)

    avg_infid_set_PF1 = average_infidelity(UPF1, exactU, backend=backend, num_shots=num_shots)
    avg_infid_set_PF2 = average_infidelity(UPF2, exactU, backend=backend, num_shots=num_shots)
    avg_infid_set_CPF1 = average_infidelity(UCPF1, exactU, backend=backend, num_shots=num_shots)
    avg_infid_set_CPF2 = average_infidelity(UCPF2, exactU, backend=backend, num_shots=num_shots)


    list_avg_infid_PF1.append(np.mean(avg_infid_set_PF1))
    list_std_infid_PF1.append(np.std(avg_infid_set_PF1))

    list_avg_infid_PF2.append(np.mean(avg_infid_set_PF2))
    list_std_infid_PF2.append(np.std(avg_infid_set_PF2))

    list_avg_infid_CPF1.append(np.mean(avg_infid_set_CPF1))
    list_std_infid_CPF1.append(np.std(avg_infid_set_CPF1))

    list_avg_infid_CPF2.append(np.mean(avg_infid_set_CPF2))
    list_std_infid_CPF2.append(np.std(avg_infid_set_CPF2))

Following are examples for depth of some quantum circuits used in hardware implementations.

In [5]:
print(f"""
Depth comparison
-----------------
Depth of Uexact: {exactU.decompose().depth()}
Depth of UPF1: {UPF1.decompose().depth()}
Depth of UPF2: {UPF2.decompose().depth()}
Depth of UCPF1: {UCPF1.decompose().depth()}
Depth of UCPF2: {UCPF2.decompose().depth()}
""")


Depth comparison
-----------------
Depth of Uexact: 23
Depth of UPF1: 40
Depth of UPF2: 41
Depth of UCPF1: 70
Depth of UCPF2: 65



Below is a visualziation for one of the circuits.

In [6]:
print(exactU.draw(fold=-1))
print(exactU.decompose().draw(fold=-1))

     ┌───┐                   ┌───┐┌──────────┐┌───┐                  ┌───┐
q_0: ┤ X ├──────o─────────■──┤ X ├┤ Rz(1.99) ├┤ X ├──■────────o──────┤ X ├
     └─┬─┘┌─────┴──────┐┌─┴─┐└─┬─┘├──────────┤└─┬─┘┌─┴─┐┌─────┴─────┐└─┬─┘
q_1: ──■──┤ Ry(-0.005) ├┤ H ├──■──┤ Rz(2.01) ├──■──┤ H ├┤ Ry(0.005) ├──■──
          └────────────┘└───┘     └──────────┘     └───┘└───────────┘     
global phase: 4.2832
     ┌───┐┌───┐              ┌───┐                                  ┌───┐┌──────────┐┌───┐                     ┌───┐                          ┌───┐┌───┐
q_0: ┤ X ├┤ X ├──────■───────┤ X ├────────────■─────────────────────┤ X ├┤ U1(1.99) ├┤ X ├─────────────────■───┤ X ├───────────────────■──────┤ X ├┤ X ├
     └─┬─┘└───┘┌─────┴──────┐├───┤┌───┐┌───┐┌─┴─┐┌─────┐┌───┐┌─────┐└─┬─┘├──────────┤└─┬─┘┌───┐┌───┐┌───┐┌─┴─┐┌┴───┴┐┌───┐┌─────┐┌─────┴─────┐└───┘└─┬─┘
q_1: ──■───────┤ Ry(-0.005) ├┤ S ├┤ H ├┤ T ├┤ X ├┤ Tdg ├┤ H ├┤ Sdg ├──■──┤ U1(2.01) ├──■──┤ S ├┤ H ├┤ T ├┤ X ├┤ Tdg ├┤ H ├┤ Sdg ├┤ Ry(0.005) ├───

## Save hardware results as a CSV file

We now save the results of hardware implementation as a csv file. The first column provides description of the parameters used such as the number of shots or perturbation parameter.

In [ ]:
# Save results as a dataframe.
data = {'description': ['nqubits=2','nsteps=10','nshots=10^5','pparam=0.1','ticks=np.linspace(0.1,1,10)',0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        'avg_infid_PF1': list_avg_infid_PF1,
        'std_infid_PF1': list_std_infid_PF1,
        'avg_infid_PF2': list_avg_infid_PF2,
        'std_infid_PF2': list_std_infid_PF2,
        'avg_infid_CPF1': list_avg_infid_CPF1,
        'std_infid_CPF1': list_std_infid_CPF1,
        'avg_infid_CPF2': list_avg_infid_CPF2,
        'std_infid_CPF2': list_std_infid_CPF2
        }

# Create DataFrame
df = pd.DataFrame(data)
# we put a descriptive title for csv file with parameters used
df.to_csv('ibm_implementation_ising_2sites_10steps_1e5shots_0.1pparam_10ticks.csv', index = False)

In [21]:
# Display the csv file of the hardwaree implementation results
file_path = '../hardware experiments/ibm_implementation_ising_2sites_10steps_1e5shots_0.1pparam_20ticks.csv'
df = pd.read_csv(file_path)
df

,description,avg_infid_PF1,std_infid_PF1,avg_infid_PF2,std_infid_PF2,avg_infid_CPF1,std_infid_CPF1,avg_infid_CPF2,std_infid_CPF2
0,nqubits=2,0.017220,0.000791,0.009197,0.000236,0.004990,0.000070,0.005222,0.000413
1,nsteps=10,0.016467,0.000482,0.009597,0.000097,0.007107,0.000391,0.007567,0.000570
2,nshots=10^5,0.016260,0.001177,0.009192,0.000129,0.005717,0.000345,0.005593,0.000158
3,pparam=0.1,0.016135,0.000617,0.009102,0.000201,0.006780,0.000320,0.006132,0.000717
4,"ticks=np.linspace(0.1,1,20)",0.016565,0.000869,0.009755,0.000203,0.006082,0.000324,0.009017,0.000486
5,0,0.016608,0.000932,0.009472,0.000137,0.009150,0.000140,0.009258,0.000259
6,0,0.016195,0.000651,0.009200,0.000292,0.009217,0.000180,0.009195,0.000236
7,0,0.016397,0.000923,0.009293,0.000311,0.009262,0.000312,0.009343,0.000395
8,0,0.016192,0.000633,0.021573,0.002109,0.009147,0.000154,0.009665,0.000329
9,0,0.015860,0.000470,0.019810,0.001923,0.008940,0.000326,0.009492,0.000215


## Hardware simulations

We use noisy and noiseless hardware simulations. For noisy simulation, we use a noisy simulator of the actula hardware we used in our experiments. The noisy simulator is "FakeQuebec", which is a 127-qubit simulator. For noiseless simulations, we use Aer simulator, which is a 29-qubit simulator.

In [3]:
from qiskit_ibm_runtime.fake_provider import FakeQuebec
from qiskit_aer import AerSimulator

So the backend we use is either FakeQuebec and AerSimulator depending on whether the simulation is noisy or noiseless.

In [4]:
#backend = FakeQuebec()
backend = AerSimulator(method='statevector')
backend.num_qubits

29

The rest is similar to hardware implementations: only the backend is different.

In [ ]:
pparam = 0.1
n, J, h = 2, pparam, 1
H, Hxx, Hz = ising_qubit_hamiltonian(n, J, h)
A, B = Hz, Hxx

num_shots = 10**6

time_ticks = np.linspace(0.1,1,20)

list_avg_infid_PF1 = []
list_std_infid_PF1 = []
list_avg_infid_PF2 = []
list_std_infid_PF2 = []

list_avg_infid_CPF1 = []
list_std_infid_CPF1 = []
list_avg_infid_CPF2 = []
list_std_infid_CPF2 = []

for t in time_ticks:
    exactU = exact_evolution_ising_qcirc(J, h, n, t)
    r = 1 #r is number of stepts (nsteps)
    tau = t/r

    UPF1 = PF_qcirc(1, A, B, tau, ppart='Hxx', reps=r)
    UPF2 = PF_qcirc(2, A, B, tau, ppart='Hxx', reps=r)

    UCPF1 = CPF_symp_qcirc(1, A, B, tau, ppart='Hxx', reps=r)
    UCPF2 = CPF_symp_qcirc(2, A, B, tau, ppart='Hxx', reps=r)

    avg_infid_set_PF1 = average_infidelity(UPF1, exactU, backend=backend, num_shots=num_shots)
    avg_infid_set_PF2 = average_infidelity(UPF2, exactU, backend=backend, num_shots=num_shots)
    avg_infid_set_CPF1 = average_infidelity(UCPF1, exactU, backend=backend, num_shots=num_shots)
    avg_infid_set_CPF2 = average_infidelity(UCPF2, exactU, backend=backend, num_shots=num_shots)


    list_avg_infid_PF1.append(np.mean(avg_infid_set_PF1))
    list_std_infid_PF1.append(np.std(avg_infid_set_PF1))

    list_avg_infid_PF2.append(np.mean(avg_infid_set_PF2))
    list_std_infid_PF2.append(np.std(avg_infid_set_PF2))

    list_avg_infid_CPF1.append(np.mean(avg_infid_set_CPF1))
    list_std_infid_CPF1.append(np.std(avg_infid_set_CPF1))

    list_avg_infid_CPF2.append(np.mean(avg_infid_set_CPF2))
    list_std_infid_CPF2.append(np.std(avg_infid_set_CPF2))

In [ ]:
# Save results as a dataframe.
data = {'description': ['nqubits=2','nsteps=1','nshots=10^6', 'pparam=0.1', 'ticks=np.linspace(0.1,1,20)',0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        'avg_infid_PF1': list_avg_infid_PF1,
        'std_infid_PF1': list_std_infid_PF1,
        'avg_infid_PF2': list_avg_infid_PF2,
        'std_infid_PF2': list_std_infid_PF2,
        'avg_infid_CPF1': list_avg_infid_CPF1,
        'std_infid_CPF1': list_std_infid_CPF1,
        'avg_infid_CPF2': list_avg_infid_CPF2,
        'std_infid_CPF2': list_std_infid_CPF2
        }

# Create DataFrame
df = pd.DataFrame(data)
df.to_csv('noiseless_simulation_ising_2sites_1step_1e6shots_0.1pparam_20ticks.csv', index = False)
print(df)

In [22]:
# Display the csv file of the noiseless simulation results
file_path = '../hardware experiments/ibm_noiseless_simulation_ising_4sites_1step_1e6shots_0.1pparam_20ticks.csv'
df = pd.read_csv(file_path)
df

,description,avg_infid_PF1,std_infid_PF1,avg_infid_PF2,std_infid_PF2,avg_infid_CPF1,std_infid_CPF1,avg_infid_CPF2,std_infid_CPF2
0,nqubits=4,0.000009,0.000006,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,nsteps=1,0.000035,0.000020,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,nshots=10^6,0.000107,0.000051,8.125000e-07,9.499178e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,pparam=0.1,0.000260,0.000134,1.562500e-06,1.273222e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,"ticks=np.linspace(0.1,1,20)",0.000526,0.000264,4.750000e-06,3.766630e-06,6.250000e-08,2.420615e-07,0.000000e+00,0.000000e+00
5,0,0.000928,0.000466,1.262500e-05,6.460215e-06,1.875000e-07,3.903124e-07,1.875000e-07,3.903124e-07
6,0,0.001525,0.000765,2.800000e-05,1.415097e-05,2.500000e-07,5.590170e-07,0.000000e+00,0.000000e+00
7,0,0.002312,0.001151,5.431250e-05,2.746525e-05,1.375000e-06,1.866648e-06,3.125000e-07,5.829612e-07
8,0,0.003409,0.001705,1.065625e-04,5.351048e-05,2.625000e-06,1.899836e-06,2.187500e-06,1.509087e-06
9,0,0.004732,0.002347,1.686250e-04,8.191449e-05,4.687500e-06,2.866155e-06,3.562500e-06,2.344375e-06
